In [ ]:
import syft as sy
import torch

In [ ]:
alice = sy.VirtualMachine(name="alice")
alice_client = alice.get_root_client()
remote_torch = alice_client.torch

In [ ]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.fc1 = self.torch_ref.nn.Linear(100, 10)

    def forward(self, x):
        return self.fc1(x)

In [ ]:
model = SyNet(torch)
data = torch.randn(size=(1, 100))
result = model(data)
labels = torch.randn(size=(1, 10))
loss_func = torch.nn.L1Loss()
loss = loss_func(result, labels)
loss.backward()

In [ ]:
print(model.parameters()[-1].grad) # exists


model_ptr = model.send(alice_client)
data_ptr = data.send(alice_client)
labels_ptr = labels.send(alice_client)
results_ptr = model_ptr(data_ptr)
remote_loss_func = alice_client.torch.nn.L1Loss()
remote_loss = remote_loss_func(results_ptr, labels_ptr)
remote_loss.backward()


In [ ]:
print(model_ptr.parameters().get()[-1].grad) # exists
print(model_ptr.get().parameters()[-1].grad) # does not exist anymore